In [1]:
##Created by Connie Knupp after modifying ProjectTwoDashboard code
 #provided by SNHU for CS-340 Module 7

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
from dash import Dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#import from Crud module
from CRUD_pythonMod import AnimalShelter

###########################
# Data Manipulation / Model
###########################
#updated the username and password and CRUD Python module name

#initialize user variables
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.readMethod({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Grazioso Salvare’s logo
image_filename = "assets/Grazioso Salvare Logo.png" # replaced with correct image name
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    ### heading with logo- redirects to snhu if clicked
      html.Center(html.Div([
        html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height= '20%', width= '20%')],
            href='https:www.snhu.edu')
    ])),
            
     #heading identifier
    html.B(html.H3('Connie Knupp - SNHU CS-340 Dashboard', 
                               style={'textAlign': 'right', 'color': 'blue'})), 
    #horizontal line
    html.Hr(),
        
#Code for the interactive filtering options. 
    html.Div([
        html.H2(html.Label(['Filter Options: '], style={'display': 'flex', 
                                                        'float': 'left', 'color': 'blue'})),
        dcc.Dropdown(['All Animals', 'All Dogs', 'Water Rescue', 'Mountain/Wilderness Rescue', 
                     'Individual Tracking/Disaster Rescue'], 'All Animals', id= 'filter-dropdown')
    ]),
    html.Br(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         
#Features for the interactive data table to make it user-friendly for the client 
                         editable=False,
                         filter_action='native',
                         sort_action='native',
                         sort_mode='multi',
                         column_selectable=False,
                         row_selectable= 'single',
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action='native',
                         page_current=0,
                         page_size=10,
                        ),
    html.Br(),
    html.Hr(),
    
      #heading with identifier for map portion
    html.Center(html.B(html.H1('Connie Knupp - SNHU CS-340 Interactive Map', 
                                style={'textAlign': 'center', 'color': 'blue'}))
                 ),
    
    #This sets up the dashboard so that the chart and the geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    #line break
    html.Br(),
    html.Div(id='graph-id2', className='col s12 m6'),
    #logo if clicked redirects to snhu.com
    html.Center(html.Div([
        html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height= '10%', width= '10%')],
            href='https:www.snhu.edu')
    ])),
])


#############################################
# Interaction Between Components / Controller
#############################################

#Update dashboard for dropdown filters  
@app.callback(
    Output('datatable-id','data'),
    [Input('filter-dropdown', 'value')])
def update_dashboard(value):
##Code to filter interactive data table with MongoDB queries
    #Start Case
    df = pd.DataFrame.from_records(db.readMethod({}))
    
    if (value == 'All Animals'):
        df = pd.DataFrame.from_records(db.readMethod({}))
    elif (value == 'All Dogs'):
        df = pd.DataFrame.from_records(db.readMethod({'animal_type': 'Dog'}))
    elif (value == 'Water Rescue'):
        df = pd.DataFrame.from_records(db.readMethod({'animal_type': 'Dog', 'breed': {'$in': ["Labrador Retriever Mix", 
                                                                                              "Chesapeake Bay Retreiver", "Newfoundland"]},
                                                     'sex_upon_outcome': 'Intact Female', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
                                                     }))
    elif (value == 'Mountain/Wilderness Rescue'):
        df = pd.DataFrame.from_records(db.readMethod({'animal_type': 'Dog', 'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 
                                                                                              'Old English Sheepdog', 'Siberian Husky',
                                                                                              'Rottweiler']},
                                                     'sex_upon_outcome': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
                                                     }))
    elif (value == 'Individual Tracking/Disaster Rescue'):
        df = pd.DataFrame.from_records(db.readMethod({'animal_type': 'Dog', 'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 
                                                                                              'Golden Retriever', 'Bloodhound', 'Rottweiler']},
                                                     'sex_upon_outcome': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
                                                     }))
    #columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    
    
    # MongoDB v5+ is going to return the '_id' column and that is going to have an 
    # invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
    # it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
    # inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
    df.drop(columns=['_id'], inplace=True)
    data = df.to_dict('records')
    
    return data

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback([Output('graph-id', "children"),
               Output('graph-id2', "children")],
     [Input('datatable-id', "derived_virtual_data")],
             prevent_initial_call=True)
def update_graphs(viewData):
    if viewData is None:
        graphData = {'breed': 'Gathering Data or No Matches'}
    else:
        graphData= pd.DataFrame.from_dict(viewData)
    figure1 = dcc.Graph(figure = px.pie(graphData, names='breed', title='Resulting Animals by Breed').update_traces
                      (textposition= 'inside').update_layout(uniformtext_minsize=11, uniformtext_mode = 'hide')), 
    figure2 = dcc.Graph(figure = px.histogram(graphData, x='breed', color='breed', labels={'breed': 'Breed'},
                                            title='Breed Counts').update_layout(yaxis_title='Count'))
    #Return cleaned up pie chart and histogram
    return [ figure1, figure2]
    
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")],
    prevent_initial_call=True)
def update_map(viewData, index):  

    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    if viewData is None: #Return Map Austin TX is at [30.75,-97.48]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                # Column 13 and 14 define the grid-coordinates for the map
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
                dl.Marker(position=[30.75,-97.48], children=[
                    dl.Tooltip("Austin"),
                    dl.Popup([
                        html.H1("Animal Shelter"),
                    ])
                ])
            ])
        ]
    else:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                # Column 13 and 14 define the grid-coordinates for the map
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
                dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                    dl.Tooltip(dff.iloc[row,4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row,9])
                    ])
                ])
            ])
        ]




app.run_server(debug=True)


Dash app running on http://127.0.0.1:27593/
